In [22]:
%matplotlib inline

In [23]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from scipy.constants import R
from openap import prop
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from pprint import pprint

In [24]:
# Load data with Dask
df_challenge = dd.read_csv('../data/challenge_set.csv')
df_trajectory = dd.read_parquet('../data/2022-02-0*.parquet')
# df_trajectory = dd.read_parquet('../data/2022-01-01.parquet')

## Data prep

In [25]:
# Define a function for interpolation within each partition
def interpolate_columns(df, columns):
    for column in columns:
        df[column] = df[column].interpolate()  # Pandas interpolation within each partition
    return df

# Apply the interpolation function using map_partitions
df_trajectory = df_trajectory.map_partitions(interpolate_columns, columns=['track', 'vertical_rate', 'groundspeed'])

In [26]:
# Merge DataFrames on 'flight_id'
df_combined = dd.merge(df_challenge, df_trajectory, on='flight_id', how='inner')

## Feature Encoding:

In [27]:
# Define a function to perform label encoding within each partition
def label_encode_partition(df, column):
    le = LabelEncoder()
    df[column + '_category'] =  le.fit_transform(df[column].astype(str)) + 1  # Avoid 0s in encoding
    return df

# Apply the encoding function to each partition using map_partitions
def label_encode_dask(df, column):
    return df.map_partitions(label_encode_partition, column=column)

In [28]:
# Apply the function to the desired columns
df_combined = label_encode_dask(df_combined, 'adep')
df_combined = label_encode_dask(df_combined, 'ades')
df_combined = label_encode_dask(df_combined, 'aircraft_type')
df_combined = label_encode_dask(df_combined, 'wtc')
df_combined = label_encode_dask(df_combined, 'airline')
df_combined = label_encode_dask(df_combined, 'icao24')

In [29]:
# Encode datetime columns
def encode_datetime(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)
    return data

In [30]:
df_combined['actual_offblock_time'] = dd.to_datetime(df_combined['actual_offblock_time'], utc=True)
df_combined['actual_offblock_time_month'] = df_combined["actual_offblock_time"].dt.month
df_combined['actual_offblock_time_day'] = df_combined["actual_offblock_time"].dt.day
df_combined['actual_offblock_time_hour'] = df_combined["actual_offblock_time"].dt.hour
df_combined['actual_offblock_time_minute'] = df_combined["actual_offblock_time"].dt.minute
df_combined = encode_datetime(df_combined, 'actual_offblock_time_month', 12)
df_combined = encode_datetime(df_combined, 'actual_offblock_time_day', 31)
df_combined = encode_datetime(df_combined, 'actual_offblock_time_hour', 24)
df_combined = encode_datetime(df_combined, 'actual_offblock_time_minute', 60)

df_combined['arrival_time'] = dd.to_datetime(df_combined['arrival_time'], utc=True)
df_combined['arrival_time_month'] = df_combined["arrival_time"].dt.month
df_combined['arrival_time_day'] = df_combined["arrival_time"].dt.day
df_combined['arrival_time_hour'] = df_combined["arrival_time"].dt.hour
df_combined['arrival_time_minute'] = df_combined["arrival_time"].dt.minute
df_combined = encode_datetime(df_combined, 'arrival_time_month', 12)
df_combined = encode_datetime(df_combined, 'arrival_time_day', 31)
df_combined = encode_datetime(df_combined, 'arrival_time_hour', 24)
df_combined = encode_datetime(df_combined, 'arrival_time_minute', 60)

### Barometric Formula:

The air pressure $P$ at altitude $h$ is given by:

$$
P(h) = P_0 \cdot \exp\left( \frac{-Mgh}{RT} \right)
$$

Where:

- $P(h)$ is the pressure at altitude $h$,
- $P_0$ is the pressure at sea level (standard pressure: 101325 Pa),
- $M$ is the molar mass of Earth's air (approximately $0.029 \, \text{kg/mol}$),
- $g$ is the acceleration due to gravity $(9.81 \, \text{m/s}^2)$,
- $h$ is the altitude (in meters),
- $R$ is the universal gas constant $(8.314 \, \text{J/(mol·K)})$,
- $T$ is the temperature at altitude $h$ (in Kelvin).

### Air Density Formula:

The air density $rho$ is given by the ideal gas law:

$$
\rho = \frac{P}{R_s T}
$$

Where:
- $P$ is the pressure at altitude $h$,
- $R_s$ is the specific gas constant for dry air (approximately $287.05 \, \text{J/(kg·K)}$),
- $T$ is the temperature in Kelvin.

### Combined Formula for Air Density:

$$
\rho(h) = \frac{P_0 \cdot \exp\left( \frac{-Mgh}{RT} \right)}{R_s T}
$$

In [31]:
# Constants
p0 = 101325  # Pressure at sea level (Pa)
T0 = 288.15  # Standard temperature at sea level (K)
g = 9.80665  # Acceleration due to gravity (m/s²)
L = 0.0065  # Temperature lapse rate (K/m)
R = 8.3144598  # Universal gas constant (J/(mol·K))
M = 0.0289644  # Molar mass of Earth's air (kg/mol)
R_air = 287.05  # Specific gas constant for dry air (J/(kg·K))
C_L = 0.5  # Approximate lift coefficient (this can vary, but 0.5 is a reasonable starting point for climb)

In [32]:
# Function to calculate air density with humidity
def air_density_with_humidity(altitude, temperature, specific_humidity):
    temperature_kelvin = temperature + 273.15
    
    # Calculate pressure at altitude using barometric formula
    pressure = p0 * (1 - (L * altitude) / T0) ** (g * M / (R * L))
    
    # Calculate partial pressure of water vapor (e) using specific humidity
    e = (specific_humidity * pressure) / (0.622 + specific_humidity)
    
    # Calculate virtual temperature (T_v)
    virtual_temperature = temperature_kelvin * (1 + 0.61 * specific_humidity)
    
    # Calculate air density using the virtual temperature
    density = pressure / (R_air * virtual_temperature)
    
    return density    

## Wind speed
v = $\sqrt[2]{u\_wind^2+v\_wind^2}$

In [33]:
def wind_speed(u_wind, v_wind):
    return np.sqrt(u_wind **2 + v_wind ** 2)

## True Air speed
v = $\sqrt[2]{groundspeed^2+wind\_speed^2}$

In [34]:
# Function to calculate true airspeed with altitude-based wind components
def true_airspeed(groundspeed, u_wind, v_wind):
    return np.sqrt(groundspeed**2 + wind_Speed(u_wind, v_wind))

## estimate lift force

In [35]:
df_combined['wind_speed'] = np.sqrt(df_combined['u_component_of_wind']**2 + df_combined['v_component_of_wind']**2)
df_combined['true_airspeed'] = np.sqrt(df_combined['groundspeed']**2 + df_combined['wind_speed']**2)
df_combined['pressure'] = p0 * (1 - (L * df_combined['altitude']) / T0) ** (g * M / (R * L))
df_combined['pressure_e'] = (df_combined['specific_humidity'] * df_combined['pressure']) / (0.622 + df_combined['specific_humidity'])
df_combined['virtual_temperature'] = (df_combined['temperature'] + 273.15) * (1 + 0.61 * df_combined['specific_humidity'])
df_combined['air_density'] = df_combined['pressure'] / (R_air * df_combined['virtual_temperature'])
# Assume K is some constant value based on typical aircraft performance
# (Note: This needs to be estimated or assumed. We'll use a placeholder value for now)
K = 0.5  # A reasonable placeholder value for K based on aircraft type
df_combined['lift_force'] = K * df_combined['air_density'] * df_combined['true_airspeed']**2

In [36]:
df_combined = df_combined.compute()

## Splitting Data

In [37]:
feature_names = [
    'adep_category',
    'ades_category',
    'wtc_category',
    'airline_category',
    'flight_duration',
    'taxiout_time',
    'flown_distance', 
    'latitude',
    'longitude',
    'altitude',
    'track',
    'icao24_category',
    'actual_offblock_time_hour',
    'actual_offblock_time_minute',
    'actual_offblock_time_hour_sin',
    'actual_offblock_time_hour_cos',
    'actual_offblock_time_minute_sin',
    'actual_offblock_time_minute_cos',
    'arrival_time_hour',
    'arrival_time_minute',
    'arrival_time_hour_sin',
    'arrival_time_hour_cos',
    'arrival_time_minute_sin',
    'arrival_time_minute_cos',
    'aircraft_type_category',
    'pressure_e',
    'air_density',
    'lift_force'
]
X, y= df_combined[feature_names], df_combined['tow']

In [38]:
len(X.columns)

28

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [40]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both train and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model

In [41]:
import lightgbm as lgb
from sklearn.metrics import root_mean_squared_error, r2_score


model = lgb.LGBMRegressor(force_row_wise=true)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print("model score", model.score(X_train_scaled, y_train))
# 0.9863556751160256

print("rmse:", root_mean_squared_error(y_test, y_pred))
print("r2 score", r2_score(y_test, y_pred))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.558863 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3312
[LightGBM] [Info] Number of data points in the train set: 29227105, number of used features: 28
[LightGBM] [Info] Start training from score 103152.011483
model score 0.9977952172955474
rmse: 14566.015810831086
r2 score 0.9607153443240901


In [42]:

# Obtain gain feature importance
gain_importance = model.feature_importances_

gain_importance_df = pd.DataFrame({'Feature': feature_names, 'Gain': gain_importance})
print(gain_importance_df.sort_values(by='Gain', ascending=False))

                            Feature  Gain
24           aircraft_type_category   453
6                    flown_distance   316
0                     adep_category   246
1                     ades_category   227
4                   flight_duration   201
3                  airline_category   199
11                  icao24_category   154
5                      taxiout_time   134
22          arrival_time_minute_sin    96
13      actual_offblock_time_minute    93
17  actual_offblock_time_minute_cos    79
12        actual_offblock_time_hour    77
2                      wtc_category    71
19              arrival_time_minute    69
18                arrival_time_hour    67
21            arrival_time_hour_cos    66
23          arrival_time_minute_cos    59
15    actual_offblock_time_hour_cos    56
7                          latitude    51
16  actual_offblock_time_minute_sin    49
20            arrival_time_hour_sin    47
26                      air_density    41
14    actual_offblock_time_hour_si

model score 0.9966796627121636
rmse: 13645.14220144024
r2 score 0.9588845971663306


model score 0.9977952172955474
rmse: 14566.015810831086
r2 score 0.9607153443240901

In [43]:
df_submission = dd.read_csv('../data/final_submission_set.csv')
df_submission

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
npartitions=1,,,,,,,,,,,,,,,,,,
,int64,string,string,string,string,string,string,string,string,string,string,string,string,string,int64,int64,int64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [44]:
# Merge DataFrames on 'flight_id'
df_submission_combined = dd.merge(df_submission, df_trajectory, on='flight_id', how='inner')
df_submission_combined

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow,timestamp,latitude,longitude,altitude,groundspeed,track,vertical_rate,icao24,u_component_of_wind,v_component_of_wind,temperature,specific_humidity
npartitions=9,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,string,string,string,string,string,string,string,string,string,string,string,string,string,int64,int64,int64,float64,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [47]:
s = df_submission_combined.compute()

In [57]:
s

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,longitude,altitude,groundspeed,track,vertical_rate,icao24,u_component_of_wind,v_component_of_wind,temperature,specific_humidity
0,249278240,2022-02-01,fe62bdf52d98dfe6a942dc9257f57b6e,KMIA,Miami,US,LEMD,Madrid Barajas,ES,2022-02-01T00:08:00Z,...,-10.628226,37000.0,499.666667,117.875408,-1066.666667,249278240,-6.628430,-1.206263,212.529274,0.000016
1,249278240,2022-02-01,fe62bdf52d98dfe6a942dc9257f57b6e,KMIA,Miami,US,LEMD,Madrid Barajas,ES,2022-02-01T00:08:00Z,...,-10.628226,37000.0,483.333333,112.922274,-533.333333,249278240,-6.628242,-1.206366,212.529302,0.000016
2,249278240,2022-02-01,fe62bdf52d98dfe6a942dc9257f57b6e,KMIA,Miami,US,LEMD,Madrid Barajas,ES,2022-02-01T00:08:00Z,...,-10.628226,37000.0,467.000000,107.969140,0.000000,249278240,-6.628054,-1.206470,212.529330,0.000016
3,249278240,2022-02-01,fe62bdf52d98dfe6a942dc9257f57b6e,KMIA,Miami,US,LEMD,Madrid Barajas,ES,2022-02-01T00:08:00Z,...,-10.628226,37000.0,467.000000,107.969140,0.000000,249278240,-6.627866,-1.206573,212.529358,0.000016
4,249278240,2022-02-01,fe62bdf52d98dfe6a942dc9257f57b6e,KMIA,Miami,US,LEMD,Madrid Barajas,ES,2022-02-01T00:08:00Z,...,-10.628226,37000.0,467.000000,107.969140,0.000000,249278240,-6.627678,-1.206677,212.529386,0.000016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758525,249461780,2022-02-09,2cfa95895460bc347677ac43b596df98,KJFK,New York JFK,US,LEBL,Barcelona,ES,2022-02-09T23:49:00Z,...,1.130432,13725.0,357.000000,118.072487,-1856.000000,249461780,4.325013,6.828034,265.625560,0.000433
1758526,249461780,2022-02-09,2cfa95895460bc347677ac43b596df98,KJFK,New York JFK,US,LEBL,Barcelona,ES,2022-02-09T23:49:00Z,...,1.130432,13725.0,357.000000,118.072487,-1856.000000,249461780,4.325017,6.827911,265.625505,0.000433
1758527,249461780,2022-02-09,2cfa95895460bc347677ac43b596df98,KJFK,New York JFK,US,LEBL,Barcelona,ES,2022-02-09T23:49:00Z,...,1.130432,13725.0,357.000000,118.072487,-1856.000000,249461780,4.325020,6.827787,265.625451,0.000433
1758528,249461780,2022-02-09,2cfa95895460bc347677ac43b596df98,KJFK,New York JFK,US,LEBL,Barcelona,ES,2022-02-09T23:49:00Z,...,1.130432,13725.0,357.000000,118.072487,-1856.000000,249461780,4.325024,6.827664,265.625396,0.000433
